In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy.io import loadmat
from scipy.optimize import minimize
import pickle

In [ ]:
from bayesee.evaluation import *

In [ ]:
%load_ext autoreload
%autoreload 2
plt.style.use('bayesee.academic')

In [ ]:
repo_path = Path.cwd().parents[0]
print(repo_path)

In [ ]:
subject = "AZ_t1"
file_name = repo_path / f"data/covert-search/large-field/p1_data_{subject}.pickle"

with open(file_name, "rb") as f:
    stimulus, response = pickle.load(f)

metadata = stimulus["metadata"]
print(metadata.keys())

In [ ]:
array_dprime, array_criterion = compute_dprime_criterion_w_discrete_response(
    stimulus, response
)
array_Bb, array_Aa = compute_confusion_matrix_w_discrete_response(stimulus, response)

In [ ]:
fig, ax = plt.subplots()
amplitude_unique, index_unique = np.unique(
    stimulus["df"]["amplitude"], return_index=True
)
ax.scatter(amplitude_unique, array_dprime[index_unique], s=250, label="d'")
ax.scatter(amplitude_unique, array_criterion[index_unique], s=250, label="$\gamma$")
ax.legend(loc="best")

ax.set(xlabel="Target amplitude", ylabel="Value")

In [ ]:
x0 = [1, 1, 0]

results = minimize(
    negative_loglikelihood_w_parameter,
    x0,
    args=(stimulus, response, unit_likelihood_abc),
)

In [ ]:
a, b, c = results.x
fig, ax = plt.subplots()
amplitude_unique = np.unique(stimulus["df"]["amplitude"])

ax.scatter(amplitude_unique, array_Bb[index_unique], s=250, label="Hit rate")
ax.scatter(amplitude_unique, array_Aa[index_unique], s=250, label="CR rate")

ax.plot(
    amplitude_unique,
    norm.cdf(0.5 * (amplitude_unique / a) ** b - c),
    label="Hit Rate Fit",
)

ax.plot(
    amplitude_unique,
    norm.cdf(0.5 * (amplitude_unique / a) ** b + c),
    label="CR Rate Fit",
)

ax.legend(loc="best")